In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape

# Part A: Input file handling

## Read all Epicentres / remove unuseful data / combine into single csv file

### Read epicenters from single CSV file

In [ ]:
df = pd.read_excel('./Epicenter/CPTI15_v2.0.xlsx',sheet_name='catalogue',encoding='latin1')

## Remove NaN, duplicated events
df.dropna(subset=['LatDef','LonDef'],inplace=True)
df = df.drop_duplicates(subset=['Year','Mo','Da','Ho','Mi','Se','LatDef','LonDef'], keep='first')

## Create dataframe with only Lat & Lon & Mw
df_LL = df[['LatDef']+['LonDef']+['MwDef']]
df_LL.columns = ['Lat', 'Lon', 'Mw']

## Save to single csv file
df_LL.to_csv('A_single.csv',index=False,encoding='utf-8')

### Read epicenters from multiple CSV files

In [ ]:
import glob
df = pd.concat([pd.read_csv(f,encoding='latin1') for f in glob.glob('./Epicenter/*.csv')],ignore_index = True)

## Remove NaN, duplicated events
df.dropna(subset=['Lat','Lon'],inplace=True) # *.csv
df = df.drop_duplicates(subset=['Year Mo Da Ho Mi Se','LatDef','LonDef'], keep='first')

## Create dataframe with only Lat & Lon & Mw
df_LL = df[['Lat']+['Lon']+['Mw']]

## Save to single csv file
df_LL.to_csv('A_multiple.csv',index=False,encoding='utf-8')

## Match Lat Lon points to geojson data / store inside new csv file

### Regional

In [ ]:
## Epicenter input & Filter by magnitude
df_LL = pd.read_csv('A_single.csv')
df_LL = df_LL[(df_LL['Mw'] >= 5) & (df_LL['Mw'] <= 7)] # Magnitude filter better early to reduce work later

## GeoJson file
json_dir='geojson-italy-master/geojson'
json_file='IT_regions.json'
## Output file
output_dir='.'
output_file='region.csv'

JsonFile=json_dir+'/'+json_file
OutFile=output_dir+'/'+output_file

## Initialise dataframe
df_data = pd.DataFrame(index=range(df_LL.shape[0]),columns=['Lat','Lon','Mw','reg_name','reg_istat'])
df_data = df_data.astype({'Lat':float,'Lon':float,'Mw':float,'reg_name':str,'reg_istat':str})

## Populate df_data
region = gpd.read_file(JsonFile)
for i in range(df_LL.shape[0]):
    point=Point(df_LL.iloc[i,1],df_LL.iloc[i,0])
    for poly,reg,Ristat in zip(region['geometry'],region['reg_name'],region['reg_istat_code_num']):
        if shape(poly).contains(point):
            df_data.loc[i]=pd.Series({'Lat':df_LL.iloc[i,0],'Lon':df_LL.iloc[i,1],'Mw':df_LL.iloc[i,2],'reg_name':reg,'reg_istat':Ristat})
            break
            
## Clean & Sort
df_data.dropna(subset=['Lat','Lon'],inplace=True) # Incase epicentre has no assigned geojson area (Sea)
df_data.sort_values(by=['reg_name'],inplace=True)

## Filter by region(s)
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo')] # Single region
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo') | (df_data['reg_name'] == 'Calabria')] # Multiple regions

## Format & Write to CSV file
df_data['Lat'] = df_data['Lat'].map(lambda x: '%2.3f' % x)
df_data['Lon'] = df_data['Lon'].map(lambda x: '%2.3f' % x)
df_data.to_csv(OutFile,index=False,encoding='utf-8')

### Provincial

In [ ]:
## Epicenter input & Filter by magnitude
df_LL = pd.read_csv('A_single.csv')
df_LL = df_LL[(df_LL['Mw'] >= 5) & (df_LL['Mw'] <= 7)] # Magnitude filter better early to reduce work later

## GeoJson file
json_dir='geojson-italy-master/geojson'
json_file='IT_provinces.json'
## Output file
output_dir='.'
output_file='province.csv'

JsonFile=json_dir+'/'+json_file
OutFile=output_dir+'/'+output_file

## Initialise combined dataframe
df_data = pd.DataFrame(index=range(df_LL.shape[0]),columns=['Lat','Lon','Mw','reg_name','reg_istat','prov_name','prov_istat'])
df_data = df_data.astype({'Lat':float,'Lon':float,'Mw':float,'reg_name':str,'reg_istat':str,'prov_name':str,'prov_istat':str})

## Populate df_data
province = gpd.read_file(JsonFile)
for i in range(df_LL.shape[0]):
    point=Point(df_LL.iloc[i,1],df_LL.iloc[i,0])
    for poly,reg,Ristat,prov,Pistat in zip(province['geometry'],province['reg_name'],province['reg_istat_code_num'],province['prov_name'],province['prov_istat_code_num']):
        if shape(poly).contains(point):
            df_data.loc[i]=pd.Series({'Lat':df_LL.iloc[i,0],'Lon':df_LL.iloc[i,1],'Mw':df_LL.iloc[i,2],'reg_name':reg,'reg_istat':Ristat,'prov_name':prov,'prov_istat':Pistat})
            break

## Clean & Sort
df_data.dropna(subset=['Lat','Lon'],inplace=True) # Incase epicentre has no assigned geojson area (Sea)
df_data.sort_values(by=['reg_name','prov_name'],inplace=True)

## Filter by region(s) / province(s)
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo')] # Single region
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo') | (df_data['reg_name'] == 'Calabria')] # Multiple regions
#df_data = df_data[(df_data['prov_name'] == 'Benevento')] # Single province
#df_data = df_data[(df_data['prov_name'] == 'Benevento') | (df_data['prov_name'] == "L'Aquila")] # Multiple provinces

## Format & Write to CSV file
df_data['Lat'] = df_data['Lat'].map(lambda x: '%2.3f' % x)
df_data['Lon'] = df_data['Lon'].map(lambda x: '%2.3f' % x)
df_data.to_csv(OutFile,index=False,encoding='utf-8')

### Municipal

In [ ]:
## Epicenter input & Filter by magnitude
df_LL = pd.read_csv('A_single.csv')
df_LL = df_LL[(df_LL['Mw'] >= 5) & (df_LL['Mw'] <= 7)] # Magnitude filter better early to reduce work later

## GeoJson file
json_dir='geojson-italy-master/geojson'
json_file='IT_municipalities.json'
## Output file
output_dir='.'
output_file='municipality.csv'

JsonFile=json_dir+'/'+json_file
OutFile=output_dir+'/'+output_file

## Initialise combined dataframe
df_data = pd.DataFrame(index=range(df_LL.shape[0]),columns=['Lat','Lon','Mw','reg_name','reg_istat','prov_name','prov_istat','munic_name'])
df_data = df_data.astype({'Lat':float,'Lon':float,'Mw':float,'reg_name':str,'reg_istat':str,'prov_name':str,'prov_istat':str,'munic_name':str})

## Populate df_data
municipality = gpd.read_file(JsonFile)
for i in range(df_LL.shape[0]):
    point=Point(df_LL.iloc[i,1],df_LL.iloc[i,0])
    for poly,reg,Ristat,prov,Pistat,munic in zip(municipality['geometry'],municipality['reg_name'],municipality['reg_istat_code_num'],municipality['prov_name'],municipality['prov_istat_code_num'],municipality['name']):
        if shape(poly).contains(point):
            df_data.loc[i]=pd.Series({'Lat':df_LL.iloc[i,0],'Lon':df_LL.iloc[i,1],'Mw':df_LL.iloc[i,2],'reg_name':reg,'reg_istat':Ristat,'prov_name':prov,'prov_istat':Pistat,'munic_name':munic})
            break

## Clean & Sort
df_data.dropna(subset=['Lat','Lon'],inplace=True) # Incase epicentre has no assigned geojson area (Sea)
df_data.sort_values(by=['reg_name','prov_name','munic_name'],inplace=True)

## Filter by region(s) / province(s) / municipality(s)
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo')] # Single region
#df_data = df_data[(df_data['reg_name'] == 'Abruzzo') | (df_data['reg_name'] == 'Calabria')] # Multiple regions
#df_data = df_data[(df_data['prov_name'] == 'Benevento')] # Single province
#df_data = df_data[(df_data['prov_name'] == 'Benevento') | (df_data['prov_name'] == "L'Aquila")] # Multiple provinces
#df_data = df_data[(df_data['munic_name'] == 'Celano')] # Single municipalty
#df_data = df_data[(df_data['munic_name'] == 'Celano') | (df_data['munic_name'] == 'Sortino')] # Multiple municipalities

## Format & Write to CSV file
df_data['Lat'] = df_data['Lat'].map(lambda x: '%2.3f' % x)
df_data['Lon'] = df_data['Lon'].map(lambda x: '%2.3f' % x)
df_data.to_csv('municipality.csv',index=False,encoding='utf-8')